In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Spam SMS Collection', sep='\t', names=['label', 'message'])

In [3]:
df.shape

(5572, 2)

In [4]:
#Data Cleaning and Preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Krunal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
  # Cleaning special character from the message
  message = re.sub(pattern='[^a-zA-Z]', repl=' ', string=df.message[i])
  # Converting the entire message into lower case
  message = message.lower()
   # Tokenizing the message by words
  words = message.split()
  # Removing the stop words
  words = [word for word in words if word not in set(stopwords.words('english'))]
  # Stemming the words
  words = [ps.stem(word) for word in words]
  # Joining the stemmed words
  message = ' '.join(words)
  # Building a corpus of messages
  corpus.append(message)


In [6]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)
X = cv.fit_transform(corpus).toarray()

# Extracting dependent variable from the dataset
y = pd.get_dummies(df['label'])
y = y.iloc[:, 1].values

In [7]:
import pickle
# Creating a pickle file for the CountVectorizer
pickle.dump(cv, open('cv-transform.pkl', 'wb'))

# Model Building

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


In [8]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.3)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Creating a pickle file for the Multinomial Naive Bayes model
filename = 'spam-sms-mnb-model.pkl'
pickle.dump(classifier, open(filename, 'wb'))

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)

In [10]:
accuracy

0.9883408071748879